<a href="https://colab.research.google.com/github/abhishektripathi66/RecomendationSystem/blob/main/nextwordclearrecomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import nltk
from nltk.tokenize import RegexpTokenizer
import json
import random
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

In [ ]:
with open("pizza.txt", "r") as file:   # Unpickling
    cleaned_text = file.read()
# with open("pizza.txt", "r") as file:   # Unpickling
#     cleaned_twitter = file.read()
cleaned_corpus=cleaned_text
print(len(cleaned_corpus))

In [ ]:
def create_data(length):
    seq=[]
    start=random.randint(0, len(cleaned_corpus)) # Used half of the corpus due to memory error
    for i in range(length,len(cleaned_corpus)):
        words = cleaned_corpus[i-length+start:i+start]
        print(words)
        line = ''.join(words)
        if line.strip()!='':
          seq.append(line)
    with open('len'+str(length)+'.json', 'w') as fp:
        json.dump(seq, fp)

    del seq

In [ ]:
#Sequence of length 2
create_data(2)

In [ ]:
create_data(4)

In [ ]:
create_data(7)

In [ ]:
pip install numpy --upgrade

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pickle
import numpy as np
def encoding_data(length):
    with open('len'+str(length)+'.json', 'r') as fp:
        seq=json.load( fp)
    print(seq)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(seq)
    sequences = tokenizer.texts_to_sequences(seq)
    # print(sequences)
    length = max(map(len, sequences))
    sequences=np.array([xi+[0]*(length-len(xi)) for xi in sequences])
    # sequences = np.array([np.array(xi) for xi in sequences])
    # flat_array = [item for sublist in sequences for item in sublist]
    # sequences = flat_array
    # sequences=np.array(sequences)
    # print(sequences)
    vocab=len(tokenizer.word_counts)+1
    data_x=sequences[:,-1]
    print(data_x)
    data_y=sequences[:,0]
    print(data_y)
    print(type(data_y))
    data_y = to_categorical(data_y, num_classes=vocab)
    words_to_index = tokenizer.word_index
    with open('tokenizer_len'+str(length)+'.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    del seq
    return data_x,data_y,vocab,words_to_index

In [ ]:
#Attention Model for classification
#Code Reference below:
#https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/#:~:text=Let's%20not%20implement%20a%20simple,Keras%20custom%20layer%20generation%20rule.
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
class attention(Layer):
    def init(self):
        super(attention,self).__init__()
    def build(self,input_shape):
        self.W=self.add_weight(name='att_weight',shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name='att_bias',shape=(input_shape[-2],1),initializer="zeros")
        super(attention, self).build(input_shape)
    def call(self,x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        a = K.softmax(e, axis=1)
        output = x*a
        return K.sum(output, axis=1)

In [ ]:
def lstm_model(length,unit1,n):
    import matplotlib.pyplot as plt
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input,Embedding,Bidirectional
    from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,LearningRateScheduler,EarlyStopping
    from tensorflow.keras.models import Model,Sequential,save_model
    from tensorflow.keras.optimizers import Adam
    # Calling the encoding function to get the data of specified length and the vocabulary
    data_x,data_y,v,wti=encoding_data(length)
    print("Data Encoded")
    print("Data_x",data_x[:5])
    print("Data_y",data_y[:5])
    print("Vocab_Size",v)

    # Preparing the model based on the inputs of unit1 ,unit2 and vocab values
    model = Sequential()
    #model.add(Embedding(input_dim=v, output_dim=300 , input_length=length-1, weights = [embed_matrix], trainable=False))
    model.add(Embedding(v, length-1, input_length=length-1))
    model.add(Bidirectional(LSTM(unit1, return_sequences=True)))
    model.add(attention())
    #model.add(Dropout(0.5))
    #model.add(Dense(50, activation='relu'))
    model.add(Dense(v, activation='softmax'))
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    filepath="lstmatt_len"+str(length)+".hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    '''
    def scheduler(epoch):

        if epoch < 60:
            return 0.001
        elif epoch < 100:
            return 0.0005
        else:
            return 0.0001
    '''
    #lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=3, min_lr=0.0001,min_delta=0.5,verbose=1)
    #es = EarlyStopping(monitor='val_loss', patience=5,min_delta=0.1,verbose=1)
    callbacks_list = [checkpoint]
    # fit model with epochs 130

    history=model.fit(data_x, data_y,batch_size=128, epochs=n,callbacks=callbacks_list)

    del data_x,data_y,v,wti

    # Plotting the loss and accuracy achieved by the model in each epoch
    # fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))
    # ax[0].plot(history.history['loss'])
    # ax[1].plot(history.history['accuracy'])
    # ax[0].set_title('model loss')
    # ax[1].set_title('model Accuracy')
    # ax[0].set_ylabel('loss')
    # ax[0].set_xlabel('epoch')
    # ax[1].set_ylabel('accuracy')
    # ax[1].set_xlabel('epoch')
    # fig.tight_layout(pad=10.0)
    # plt.show()

In [ ]:
#Here the history is of length 1 ,Bidirectional LSTM layer has 300 units and LSTM layer has 100 units
lstm_model(2,32,60)

In [ ]:
#Here the history is of length 3 ,Bidirectional LSTM layer has 250 units and LSTM layer has 100 units
# lstm_model(4,64,80)

In [ ]:
#Here the history is of length 6 ,Bidirectional LSTM layer has 150 units and LSTM layer has 50 units
# lstm_model(7,64,70)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pickle
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
'''
start - random integer that indicates the starting index of tokens in the test data
temp_length - length of each line in the test corpus from which test data will be created
length - length of history + next word in the test data
'''
def create_testdata2(start,temp_length,length):
    seq=[]

    for i in range(temp_length,50000):
        words = cleaned_corpus[i-temp_length+start:i+start]
        line = ' '.join(words)
        seq.append(line)
        if i % 10000==0:
            print(i , 'tokens done')
    with open('tokenizer_len'+str(length)+'.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)


    sequences = tokenizer.texts_to_sequences(seq)
    sequences = pad_sequences(sequences, maxlen=length, truncating='pre')
    sequences=np.array(sequences)

    vocab=len(tokenizer.word_counts)+1
    print(vocab)
    data_x=sequences[:,:-1]
    data_y=sequences[:,-1]
    data_y = to_categorical(data_y, num_classes=vocab)
    del seq,sequences
    return data_x,data_y


In [ ]:
#Function for generation data for error analysis
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
'''
start - random integer that indicates the starting index of tokens in the test data
temp_length - length of each line in the test corpus from which test data will be created
length - length of history + next word in the test data
'''
def create_errordata(start,temp_length,length):
    seq=[]

    for i in range(temp_length,100):
        words = cleaned_corpus[i-temp_length+start:i+start]
        line = ' '.join(words)
        seq.append(line)

    with open('tokenizer_len'+str(length)+'.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)


    sequences = tokenizer.texts_to_sequences(seq)
    sequences = pad_sequences(sequences, maxlen=length, truncating='pre')
    sequences=np.array(sequences)

    vocab=len(tokenizer.word_counts)+1
    print(vocab)
    data_x=sequences[:,:-1]
    data_y=sequences[:,-1]
    data_y = to_categorical(data_y, num_classes=vocab)
    del seq,sequences
    return data_x,data_y


In [ ]:
###Error Analysis funtion
def error_analysis(model,x,y,length):
    pred_y=model.predict(x)
    error=np.zeros(len(y))
    for i in range(len(pred_y)):
        for j in range(len(pred_y[i])):
            error[i] += -y[i][j]*np.log(pred_y[i][j])


    with open('tokenizer_len'+str(length)+'.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    x_words=tokenizer.sequences_to_texts(x)

    error_df =pd.DataFrame({'x':x_words, 'error':error})
    error_df=error_df.sort_values(by='error',ascending=False)
    return error_df

In [ ]:
#This function draws some plots using the input text of worst and best predicted data points.
def eda_on_error(error_df):
    from collections import Counter
    import seaborn as sns
    error_df['x']=error_df['x'].astype('str')
    worst_predicted=error_df.loc[error_df['error']>8]
    best_predicted=error_df.loc[error_df['error']<=8]
    wordcloud_worst = WordCloud().generate(' '.join(worst_predicted['x']))
    wordcloud_best = WordCloud().generate(' '.join(best_predicted['x']))
    worst_words = ' '.join(worst_predicted['x']).split(' ')
    best_words = ' '.join(best_predicted['x']).split(' ')

    worst_counts= Counter(worst_words)
    best_counts= Counter(best_words)
    fig, axs = plt.subplots(1, 2,figsize=(20,5),gridspec_kw={'width_ratios': [1,2]})

    axs[0].imshow(wordcloud_worst, interpolation='bilinear')
    axs[0].set_title('Wordcloud of worst input words')
    labels, values = zip(*worst_counts.items())
    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    sns.barplot(labels[:10],values[:10],ax=axs[1])
    axs[1].set_title('Histogram of top 10 worst input words')
    axs[1].set_xlabel('Words')
    axs[1].set_ylabel('Frequency')
    plt.show()


    fig, axs = plt.subplots(1, 2,figsize=(20,5),gridspec_kw={'width_ratios': [1,2]})
    axs[0].imshow(wordcloud_best, interpolation='bilinear')
    axs[0].set_title('Wordcloud of best input words')
    labels, values = zip(*best_counts.items())
    indSort = np.argsort(values)[::-1]
    labels = np.array(labels)[indSort]
    values = np.array(values)[indSort]
    sns.barplot(labels[:10],values[:10],ax=axs[1])
    axs[1].set_title('Histogram of top 10 best input words')
    axs[1].set_xlabel('Words')
    axs[1].set_ylabel('Frequency')
    plt.show()


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
# file="lstmatt_len7.hdf5"
# model_len7 = load_model(file, custom_objects={'attention': attention})
# model_len7.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# file="lstmatt_len4.hdf5"
# model_len4 = load_model(file, custom_objects={'attention': attention})
# model_len4.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

file="lstmatt_len2.hdf5"
model_len2 = load_model(file , custom_objects={'attention': attention})
model_len2.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [ ]:
# Setting the start and temp_length
start=random.randint(1, 1000)
temp_length=random.randint(7, 10)
print(start,temp_length)

In [ ]:
len2_x,len2_y=create_testdata2(start,temp_length,2)
# model_len2.evaluate(x=len2_x,y=len2_y)
del len2_x,len2_y

In [ ]:
# len4_x,len4_y=create_testdata2(start,temp_length,4)
# model_len4.evaluate(x=len4_x,y=len4_y)
# del len4_x,len4_y

In [ ]:
# len7_x,len7_y=create_testdata2(start,temp_length,7)
# model_len7.evaluate(x=len7_x,y=len7_y)
# del len7_x,len7_y

In [ ]:
len2_x,len2_y=create_errordata(start,temp_length,2)
len2_error=error_analysis(model_len2,len2_x,len2_y,2)
del len2_x,len2_y
len2_error.head(20)

In [ ]:
# eda_on_error(len2_error)

In [ ]:
# len4_x,len4_y=create_errordata(start,temp_length,4)
# len4_error=error_analysis(model_len4,len4_x,len4_y,4)
# del len4_x,len4_y
# len4_error.head(50)

In [ ]:
# eda_on_error(len4_error)

In [ ]:
# len7_x,len7_y=create_errordata(start,temp_length,7)
# len7_error=error_analysis(model_len7,len7_x,len7_y,7)
# del len7_x,len7_y
# len7_error.head(50)

In [ ]:
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def extra_space(text):
    new_text= re.sub("\s+"," ",text)
    return new_text
def sp_charac(text):
    new_text=re.sub("[^0-9A-Za-z ]", "" , text)
    return new_text
def tokenize_text(text):
    new_text=word_tokenize(text)
    return new_text

In [ ]:
def predict_next():
    import tensorflow as tf
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.models import load_model
    from tensorflow.keras.optimizers import Adam
    import pickle
    import time

    # with open('tokenizer_len7.pickle', 'rb') as handle:
    #     tokenizer_len7 = pickle.load(handle)

    # with open('tokenizer_len4.pickle', 'rb') as handle:
    #     tokenizer_len4 = pickle.load(handle)
    with open('tokenizer_len2.pickle', 'rb') as handle:
        tokenizer_len2 = pickle.load(handle)

    # file="lstmatt_len7.hdf5"
    # model_len7 = load_model(file ,custom_objects={'attention': attention})
    # model_len7.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

    # file="lstmatt_len4.hdf5"
    # model_len4 = load_model(file, custom_objects={'attention': attention})
    # model_len4.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

    file="lstmatt_len2.hdf5"
    model_len2 = load_model(file , custom_objects={'attention': attention})
    model_len2.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    while(True):
        text=input()
        start= time.time()
        cleaned_text=extra_space(text)
        cleaned_text=sp_charac(cleaned_text)
        tokenized=tokenize_text(cleaned_text)

        line = ' '.join(tokenized)


        if len(tokenized)==1:

            encoded_text = tokenizer_len2.texts_to_sequences([line])
            pad_encoded = pad_sequences(encoded_text, maxlen=1, truncating='pre')

            for i in (model_len2.predict(pad_encoded)[0]).argsort()[-3:][::-1]:


                pred_word = tokenizer_len2.index_word[i]
                print("Next word suggestion:",pred_word)
        # elif len(tokenized)<4:
        #     encoded_text = tokenizer_len4.texts_to_sequences([line])
        #     pad_encoded = pad_sequences(encoded_text, maxlen=3, truncating='pre')

        #     for i in (model_len4.predict(pad_encoded)[0]).argsort()[-3:][::-1]:


        #         pred_word = tokenizer_len4.index_word[i]
        #         print("Next word suggestion:",pred_word)
        # else:
        #     encoded_text = tokenizer_len7.texts_to_sequences([line])
        #     pad_encoded = pad_sequences(encoded_text, maxlen=6, truncating='pre')

        #     for i in (model_len7.predict(pad_encoded)[0]).argsort()[-3:][::-1]:


        #         pred_word = tokenizer_len7.index_word[i]
        #         print("Next word suggestion:",pred_word)
        print('Time taken: ',time.time()-start)


In [ ]:
predict_next()

hi
1/1 [==============================] - 1s 875ms/step
Next word suggestion: abh
Next word suggestion: ab
Next word suggestion: is
Time taken:  0.945974588394165
abhishek
1/1 [==============================] - 0s 28ms/step
Next word suggestion: abh
Next word suggestion: ab
Next word suggestion: is
Time taken:  0.10459256172180176
abhishek is 
Time taken:  0.0003516674041748047
is
1/1 [==============================] - 0s 25ms/step
Next word suggestion: abh
Next word suggestion: is
Next word suggestion: ab
Time taken:  0.10014104843139648
wow
1/1 [==============================] - 0s 24ms/step
Next word suggestion: abh
Next word suggestion: ab
Next word suggestion: is
Time taken:  0.0868675708770752
common
1/1 [==============================] - 0s 22ms/step
Next word suggestion: abh
Next word suggestion: ab
Next word suggestion: is
Time taken:  0.08568620681762695
good
1/1 [==============================] - 0s 21ms/step
Next word suggestion: abh
Next word suggestion: ab
Next word sugge